# Maestría en Ciencia de Datos, ITAM

**Curso de Optimización 2 2021-1,"Optimización Avanzada"
Prof. Erick Palacios Moreno**

* Equipo 5:  
>MIGUEL LOPEZ  
CARLOS LOPEZ  
JOSÉ ZARATE 


## RESOLVER EL PROBLEMA DE TRAVELING SALESMAN PROBLEM (A.K.A. TSP) PARA DISTINTAS CIUDADES UTILIZANDO EL MÉTODO DE HILL CLIMBING (RANDOM-RESTART)

* Distancia Euclideana.
> Dataset: National Traveling Salesman Problems, CANADA

Para ello, realizaremos una ejecución para un subgrupo del dataset con las primeras 17 ciudades con el objetivo de realizar una _test_ estandar en tiempo razonable.

-------

### Hill Climbing 

El algoritmo de hill-climbing realiza seguimiento de un estado actual y en cada iteración se mueve al estado vecino con el valor más alto o menor, según sea el caso, es decir, se dirige en la dirección que proporciona el ascenso o descenso de mayor inclinación. Termina cuando alcanza un máximo o mínimo local. Hill climbing no mira hacia el futuro más allá de los vecinos inmediatos del estado actual. Esto se asemeja a tratar de encontrar la cima del Monte Everest en una espesa niebla mientras sufre de amnesia. Toma en cuenta que una forma de utilizar hill-climbing  consiste en utilizar el negativo de una función de coste heurística como función objetivo; que ascenderá localmente al estado con menor distancia heurística a la meta.

Los problemas que tiene Hill Climbing son:

MÁXIMA LOCAL: Un máximo local es un pico que es más alto que los estados vecinos pero menor que el máximo global. Cuando este algoritmo alcanzan las proximidades de un máximo local no podrá escapar de este, amenos que estemos tratando con un problema convexo.

CORDILLERAS: Las cordilleras son un reto para los algoritmos de hill climbing que optimizan en espacios continuos. Debido a que los algoritmos de hill climbing ajustan solo un elemento en el vector a la vez, a cada paso se va a mover en una dirección alineada con el eje. Si la función objetivo determina una cordillera estrecha que asciende en una dirección no alineada con el eje, entonces el algoritmo de hill climbing solo puede ascender la cordillera en zigzag. Si los lados de la cordillera son muy pronunciados, entonces el algoritmo puede verse forzado a realizar pasos muy pequeños mientras zigzaguea hacia una mejor posición.

MESETA: Se encuentra una meseta cuando el espacio de búsqueda es plano o lo suficientemente plano como para que el valor devuelto por la función objetivo sea indistinguible del valor devuelto para las regiones cercanas debido a la precisión utilizada por la máquina para representar la misma.

-------
**MODIFICACIONES DERIVADAS DE LA ETAPA DE EXPERIMENTACIÓN:**

Se desarrollaron diversos experimentos para analizar la eficiencia del algoritmo, la tolerancia y el punto inicial. Observamos que una de las mayores dificultades a resolver era que nuestro algoritmo era ineficiente para un númer mayor a 9 ciudades para cualquier punto inicial y tolerancia asignada (norma-distncia entre dos rutas para elegir la de mayor eficiencia). Se realizó una recodificación del código a partir de consultar otros algoritmos desarrollados por la comunidad (ver README de nuestro repo) bajo los cuales logramos tiempos sumamente eficientes incluso con más de 100 ciudades.

En una segunda etapa buscaremos optimizar el código en la siguiente práctica por medio de un análisis de perfilamiento y la consecuente reprogramación; finalmente se implementará el cómputo en paralelo. 

______________

A continuación se muestra la descripción de la instancia utilizada:

In [1]:
%%bash
lscpu

Architecture:        x86_64
CPU op-mode(s):      32-bit, 64-bit
Byte Order:          Little Endian
CPU(s):              8
On-line CPU(s) list: 0-7
Thread(s) per core:  2
Core(s) per socket:  4
Socket(s):           1
NUMA node(s):        1
Vendor ID:           GenuineIntel
CPU family:          6
Model:               85
Model name:          Intel(R) Xeon(R) Platinum 8175M CPU @ 2.50GHz
Stepping:            4
CPU MHz:             3100.044
BogoMIPS:            5000.00
Hypervisor vendor:   KVM
Virtualization type: full
L1d cache:           32K
L1i cache:           32K
L2 cache:            1024K
L3 cache:            33792K
NUMA node0 CPU(s):   0-7
Flags:               fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid aperfmperf tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt tsc_deadline_timer aes xsave avx f16c rdrand hy

In [1]:
import os
import time
import numpy as np
import pandas as pd
os.chdir("../")
from src.hill_cg.hill import best_solution
from src.hill_cg.hill_mul import parallel_hc

Enseguida ejecutamos un ejemplo donde se meustra la comparación entre nuestro código sin paralelizar contra nuestro código paralelizando.

Nuestro primer ejemplo lo corremos con 20 ciudades extraidas de nuestro dataset ejemplo, el cual contiene 4,663 ciudades canadienses, mientras que par nuestro segundo ejemplo lo coreremos con un dataset de 25 ciudades.En primer lugar correremos la función `best_solution` el cual no utiliza `Pool` para déspues correr nuestro código con computo en paralelo. 

Ambas funciones regresan los mismos tres resultados: distancia más corta encontrada, la permutación correspondiente a dicha distancia y el tiempo de ejecución.

In [2]:
# Cargamos los datos 

dat = pd.read_csv("datasets/ca4663.tsp", names = ["index","uno","dos"], sep = " ")
dat.drop(['index'],axis = 1, inplace = True)
dat.dropna(inplace = True)
dat1 = dat.to_numpy()
tsp_cities = dat1[0:20,:]

### Ejemplo 20 ciudades 

Los primeros tres parámetros que se ingresan en cada función son idénticos equivalentes, mientras que el cuarto parámetro, que para `parallel_hc` es 100 y para `best_solution` es 800,  estas son  equivalentes. Ya que está instancia tiene, como se muestra en la descripción, ocho workers posibles, en cambio `best_solution` solo corre un worker a la vez.  

In [10]:
%%time
parallel_hc(tsp_cities, 0, 1e-1, 20)

CPU times: user 3.39 ms, sys: 36 ms, total: 39.4 ms
Wall time: 1.84 s


(3420.477836065084,
 [0, 3, 8, 10, 9, 4, 2, 7, 16, 14, 19, 18, 17, 15, 11, 13, 12, 6, 1, 5, 0],
 1.6029994487762451)

In [9]:
%%time
best_solution(tsp_cities, 0, 1e-1, 160)

CPU times: user 4.24 s, sys: 3.99 ms, total: 4.25 s
Wall time: 4.24 s


(3493.4940349361523,
 [0, 3, 8, 17, 18, 19, 16, 14, 7, 9, 2, 4, 10, 15, 11, 13, 12, 6, 5, 1, 0],
 4.237102746963501)

### Ejemplo 25 ciudades

In [14]:
tsp_cities_dos = dat1[0:25,:]

In [6]:
%%time
parallel_hc(tsp_cities_dos, 0, 1e-1, 20)

CPU times: user 20.6 ms, sys: 12.3 ms, total: 32.9 ms
Wall time: 5.45 s


(3688.2628362386577,
 [0,
  1,
  5,
  6,
  21,
  12,
  24,
  13,
  11,
  15,
  10,
  17,
  18,
  22,
  19,
  20,
  23,
  16,
  14,
  9,
  7,
  4,
  2,
  8,
  3,
  0],
 5.323086738586426)

In [7]:
%%time
best_solution(tsp_cities_dos, 0, 1e-1, 160)

CPU times: user 12.1 s, sys: 0 ns, total: 12.1 s
Wall time: 12.1 s


(3668.3573639350216,
 [0,
  1,
  6,
  21,
  12,
  24,
  13,
  11,
  15,
  17,
  18,
  22,
  19,
  20,
  16,
  23,
  14,
  7,
  2,
  4,
  9,
  10,
  8,
  3,
  5,
  0],
 12.07761001586914)

### Comparación entre OR-Tools y parallel_hc

A continuación mostramos la comparación entre resultados de OR-tools de google y nuestro paquete para ejemplos de 20 y 25 ciudades tanto comparando la distancia cómo el tiempo de ejccución. 

In [15]:
pip install ortools

Note: you may need to restart the kernel to use updated packages.


In [9]:
import notebooks.google_or_tools_tsp as tsp

### Ejemplo 20 ciudades: comparación OR Tools vs parallel_hc) 

In [10]:
%%time
tsp_sol = tsp.main(tsp_cities)

Objective: 3317
Route:
 0 -> 1 -> 5 -> 6 -> 12 -> 13 -> 11 -> 15 -> 17 -> 18 -> 19 -> 16 -> 14 -> 7 -> 9 -> 2 -> 4 -> 10 -> 8 -> 3 -> 0

CPU times: user 14.5 ms, sys: 0 ns, total: 14.5 ms
Wall time: 13.5 ms


In [11]:
%%time
parallel_hc(tsp_cities, 0, 1e-1, 100)

CPU times: user 21.1 ms, sys: 22.4 ms, total: 43.6 ms
Wall time: 9.2 s


(3382.97987100559,
 [0, 3, 8, 10, 2, 4, 7, 9, 14, 16, 19, 18, 17, 15, 11, 13, 12, 6, 5, 1, 0],
 9.042816638946533)

### Ejemplo 25 ciudades: comparación OR Tools vs parallel_hc) 

In [ ]:
tsp_cities_tres = dat1[0:25,:]

In [10]:
%%time
tsp_sol = tsp.main(tsp_cities)

Objective: 3317
Route:
 0 -> 1 -> 5 -> 6 -> 12 -> 13 -> 11 -> 15 -> 17 -> 18 -> 19 -> 16 -> 14 -> 7 -> 9 -> 2 -> 4 -> 10 -> 8 -> 3 -> 0

CPU times: user 14.5 ms, sys: 0 ns, total: 14.5 ms
Wall time: 13.5 ms


In [12]:
tsp_sol = tsp.main(tsp_cities)

Objective: 9943
Route:
 0 -> 1 -> 5 -> 6 -> 11 -> 13 -> 24 -> 21 -> 12 -> 25 -> 27 -> 30 -> 35 -> 33 -> 34 -> 32 -> 31 -> 49 -> 47 -> 41 -> 38 -> 44 -> 45 -> 48 -> 39 -> 40 -> 42 -> 37 -> 26 -> 28 -> 29 -> 46 -> 43 -> 36 -> 15 -> 17 -> 18 -> 22 -> 19 -> 20 -> 23 -> 16 -> 14 -> 7 -> 9 -> 2 -> 4 -> 10 -> 8 -> 3 -> 0



### Ejemplo 50 ciudades: comparación OR Tools vs parallel_hc) 

In [12]:
tsp_sol = tsp.main(tsp_cities)

Objective: 9943
Route:
 0 -> 1 -> 5 -> 6 -> 11 -> 13 -> 24 -> 21 -> 12 -> 25 -> 27 -> 30 -> 35 -> 33 -> 34 -> 32 -> 31 -> 49 -> 47 -> 41 -> 38 -> 44 -> 45 -> 48 -> 39 -> 40 -> 42 -> 37 -> 26 -> 28 -> 29 -> 46 -> 43 -> 36 -> 15 -> 17 -> 18 -> 22 -> 19 -> 20 -> 23 -> 16 -> 14 -> 7 -> 9 -> 2 -> 4 -> 10 -> 8 -> 3 -> 0



In [8]:
prueba = parallel_hc(tsp_cities)

In [9]:
prueba

(11246.084161235227,
 [0,
  1,
  5,
  6,
  3,
  8,
  10,
  19,
  22,
  20,
  23,
  16,
  14,
  7,
  2,
  4,
  9,
  18,
  17,
  15,
  13,
  24,
  26,
  40,
  39,
  48,
  45,
  44,
  38,
  47,
  49,
  41,
  31,
  32,
  34,
  33,
  35,
  42,
  30,
  37,
  27,
  25,
  12,
  21,
  28,
  29,
  46,
  43,
  36,
  11,
  0],
 574.4132664203644)

### Conclusión 

Enocntramos que en la implementación en paralelo se reducen a menos de la mitad el tiempo entre nuestra implementacion paralelizada y no paralelizada.

En la comparación de nuestro paquete con OR-Tools los resultados para 20 ciudades encontramos que nuestro paquete comienza hacer más grandes los resultados, alejándose del valor encontrado por el áuwete de Google OR Tools. Cómo en el ejemplo con 50 ciudades el resultado muestra una 13% mayor con respecto a OR Tools y esta diferencia aumenta con el número de ciudades.

Dado que este método sólo explora algunas zonas del espacio solución debido a la gran magnitud de posibles solucionnes, a partir de 20 ciudades nuesreo paquete comienza a mostrar diferncia con los resultados encontrados por OR-Tools. 